# 1. Import the required libraries

In [7]:
# Standard code libraries
import os
import platform
import glob

import numpy as np
from numpy.matlib import repmat
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import ipywidgets as widgets

# Custom code libraries from ReSurfEMG
from resurfemg.config.config import Config
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader
from resurfemg.data_classes.data_classes import (
VentilatorDataGroup, EmgDataGroup)

%matplotlib widget

## 2. Load the ventilator and sEMG data

In [43]:
def find_files(
    path,
    file_name_regex=None,
    extension_regex=None,
    folder_levels=None,
    verbose=True):

    if not os.path.isdir(path):
        raise ValueError('Specified path cannot be found.')

    if file_name_regex is None:
        file_name_regex = '**'
    elif not isinstance(file_name_regex, str):
        raise ValueError('file_name_regex should be a str.')

    if extension_regex is None:
        extension_regex = '**'
    elif not isinstance(extension_regex, str):
        raise ValueError('extension_regex should be a str.')

    if isinstance(folder_levels, list):
        depth = len(folder_levels)
        folder_levels.append('files')
    elif folder_levels is None:
        depth = None
        folder_levels = ['files']
    else:
        raise ValueError('Provide either a list, or None as folder_levels.')

    if platform.system() == 'Windows':
        path_sep = "\\"
    else:
        path_sep = '/'

    data_pattern = os.path.join(
        path, '**'+ path_sep + file_name_regex + '.' + extension_regex)
    matching_files = glob.glob(data_pattern, recursive=True)
    files_structure = [
        file_name.replace(
            path + path_sep, "").split(path_sep)
            for file_name in matching_files
    ]

    matching_files_structure = list()
    non_matching_files_structure = list()
    for file in files_structure:
        if depth is None:
            matching_files_structure.append(path_sep.join(file))
        elif isinstance(file, str) and (depth == 0):
            matching_files_structure.append(file)
        elif isinstance(file, list) and (len(file) == depth+1):
            matching_files_structure.append(file)
        else:
            non_matching_files_structure.append(file)

    files = pd.DataFrame(matching_files_structure, columns=folder_levels)
    if verbose is True:
        print('These files did not match the provided depth:\n',
            (non_matching_files_structure))
    return files

In [44]:
config = Config()
path = config.get_directory('root_patient_data_directory')
file_name_regex = None
extension_regex = 'poly5'
folder_levels = ['patient', 'date', 'measurement']
verbose = False
# Root directory for test data

files = find_files(
    path,
    file_name_regex=file_name_regex,
    extension_regex=extension_regex,
    folder_levels=folder_levels,
    verbose=verbose)

print(files)

    patient                 date measurement                    files
0    MST001  2021-12-24_09-29-46         001  Draeger_recording.Poly5
1    MST001  2021-12-24_09-29-46         001      EMG_recording.Poly5
2    MST001  2021-12-24_09-29-46         002  Draeger_recording.Poly5
3    MST001  2021-12-24_09-29-46         002      EMG_recording.Poly5
4    MST001  2021-12-24_09-29-46         003  Draeger_recording.Poly5
..      ...                  ...         ...                      ...
775  MST017  2022-12-09_12-39-29         007      EMG_recording.Poly5
776  MST017  2022-12-09_12-39-29         008  Draeger_recording.Poly5
777  MST017  2022-12-09_12-39-29         008      EMG_recording.Poly5
778  MST017  2022-12-09_12-39-29         009  Draeger_recording.Poly5
779  MST017  2022-12-09_12-39-29         009      EMG_recording.Poly5

[780 rows x 4 columns]


In [45]:
button_list = list()
for idx, folder_level in enumerate(folder_levels[:-1]):
    if idx == 0:
        filter_files = files
    else:
        filter_files = files[np.all(np.array(
            [(files[folder_levels[_idx]] == button_list[_idx].value).values 
            for _idx in range(idx)]), 0)
        ]
    
    options = list(set(filter_files[folder_level].values))
    options.sort()
    _btn = widgets.Dropdown(  
        options=options,
        value=options[0],
        description=folder_level + ':',
        disabled=False,
    )
    
    display(_btn)
    button_list.append(_btn)

# @interact(country = countryW, city = cityW, district = districtW)
# https://stackoverflow.com/questions/48407059/multiple-dependent-widgets-dropdown-menu-on-jupyter-notebook#52713761
# button_list

Dropdown(description='patient:', options=('MST001', 'MST002', 'MST003', 'MST004', 'MST005', 'MST006', 'MST007'…

Dropdown(description='date:', options=('2021-12-24_09-29-46', '2021-12-27_09-16-26', '2021-12-29_09-02-54', '2…

Dropdown(description='measurement:', options=('001', '002', '003', '004', '005', '006', '007', '008', '009'), …

In [47]:
for idx, folder_level in enumerate(folder_levels[:-1]):
    print(button_list[idx].value)

MST005
2021-12-24_09-29-46
001
